In [ ]:
!pip install yfinance
!pip install pytz

import yfinance as yf
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
import pytz


In [ ]:
# Download MSFT stock data
msft = yf.Ticker("MSFT")
data = msft.history(period="max")

# Convert the datetime index to tz-naive
data.index = data.index.tz_localize(None)


In [ ]:
# Prepare the data
data = data[["Close"]]  # Keep only the Close prices
data = data.rename(columns={"Close": "Price"})
data["Return"] = data["Price"].pct_change()  # Calculate daily returns
data = data.dropna()  # Drop NaN rows

# Split the data into training and testing sets
X = data[["Price"]].shift(1).dropna()  # Shifted prices as features
y = data["Return"].shift(-1).dropna()  # Shifted returns as target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model on the test set
score = model.score(X_test, y_test)
print(f"Model Score: {score:.2f}")



In [ ]:
# Get the prediction date from the user
prediction_date_str = input("Enter the prediction date (YYYY-MM-DD): ")
prediction_date = datetime.strptime(prediction_date_str, "%Y-%m-%d")

# Find the last known price before the prediction date
last_known_date = max(filter(lambda x: x <= prediction_date, data.index))
last_known_price = data.loc[last_known_date, "Price"]

# Make a prediction for the next day
next_day = last_known_date + timedelta(days=1)
next_day_price = last_known_price * (1 + model.predict([[last_known_price]]))

print(f"\nPredicted price for {next_day.date()}: ${next_day_price[0]:.2f}")